In [7]:
import tensorflow as tf
import numpy as np
import os
import requests
import json

# Get Content Recommendations for a Given User ID
flask app must be running. see read me for instructions on getting flask app running.

In [88]:
# send a user to get recommendations from
USER_ID = "82216a2c7d"

url = 'http://localhost:3000/recommender'
event_data = {'user_id': USER_ID}

r = requests.post(url, json=event_data)
print(r)

# returns a ranked key:value pair of content_id: predicted_engaged_pct of top 10 recommendations
json.loads(json.loads(r.text))

<Response [200]>


{"b'a533f2f57a'": 0.67427635,
 "b'487d868f96'": 0.6728412,
 "b'73e16a0447'": 0.66920125,
 "b'a1671b9953'": 0.6617259,
 "b'639ae113f9'": 0.65509045,
 "b'c7d5f7710d'": 0.6438377,
 "b'20b0745a31'": 0.6413824,
 "b'49a9f915b5'": 0.6396617,
 "b'9f943715a3'": 0.6361379,
 "b'19e9799800'": 0.6309099}

# Dev - Initial Model Loading - Dev

In [87]:
 # Load Retrieval; can also be done in TensorFlow Serving.
    
cwd = os.getcwd()
print()
retrieval_path = os.path.join(cwd, "retrieval_model")
loaded_retrieval = tf.saved_model.load(retrieval_path)

# Pass a user id in, get top predicted movie titles back.
_ , content_ids_recommended = loaded_retrieval([USER_ID])

print(f"Recommendations: {content_ids_recommended}")

cwd = os.getcwd()
print()
ranking_path = os.path.join(cwd, "ranking_model")


loaded_ranking = tf.saved_model.load(ranking_path)

final_ratings = {}

content_ids_from_retrieval = content_ids_recommended[0]

print("content from retrieval (unsorted content_ids): ", content_ids_from_retrieval)

for content_id in content_ids_from_retrieval:
    content_id = content_id.numpy()
    final_ratings[content_id] = loaded_ranking(
        # user | content_id
        {"user_id": np.array([f"{USER_ID}"]), "content_id": np.array([content_id])}
    )

print("----------------")
print("Ranked Content")
print(f"Ratings for user {USER_ID}:")
print("content_id | score")

api_return = {}
for content_id, score in sorted(
    final_ratings.items(), key=lambda x: x[1], reverse=True
):
    print(f"{content_id}: {score}")
    api_return[f'{content_id}'] = score.numpy()[0][0]

    
api_return   


Recommendations: [[b'49a9f915b5' b'a533f2f57a' b'20b0745a31' b'487d868f96' b'19e9799800'
  b'a1671b9953' b'9f943715a3' b'25ad9f4d85' b'd7dad09669' b'73e16a0447']]

content from retrieval (unsorted content_ids):  tf.Tensor(
[b'49a9f915b5' b'a533f2f57a' b'20b0745a31' b'487d868f96' b'19e9799800'
 b'a1671b9953' b'9f943715a3' b'25ad9f4d85' b'd7dad09669' b'73e16a0447'], shape=(10,), dtype=string)
----------------
Ranked Content
Ratings for user 82216a2c7d:
content_id | score
b'73e16a0447': [[0.36834288]]
b'a1671b9953': [[0.367317]]
b'487d868f96': [[0.36708573]]
b'd7dad09669': [[0.36350384]]
b'25ad9f4d85': [[0.35258558]]
b'a533f2f57a': [[0.3521912]]
b'49a9f915b5': [[0.3467191]]
b'19e9799800': [[0.33223924]]
b'20b0745a31': [[0.3312234]]
b'9f943715a3': [[0.32872432]]


{"b'73e16a0447'": 0.36834288,
 "b'a1671b9953'": 0.367317,
 "b'487d868f96'": 0.36708573,
 "b'd7dad09669'": 0.36350384,
 "b'25ad9f4d85'": 0.35258558,
 "b'a533f2f57a'": 0.3521912,
 "b'49a9f915b5'": 0.3467191,
 "b'19e9799800'": 0.33223924,
 "b'20b0745a31'": 0.3312234,
 "b'9f943715a3'": 0.32872432}